# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

#turning off warning messages
import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto real,36.5282,-6.1901,14.23,88,100,6.17,ES,1701832123
1,1,albany,42.6001,-73.9662,0.37,88,100,1.30,US,1701831950
2,2,talnakh,69.4865,88.3972,-32.51,78,7,0.79,RU,1701832124
3,3,waitangi,-43.9535,-176.5597,14.34,80,100,3.58,NZ,1701832125
4,4,yellowknife,62.4560,-114.3525,-4.77,100,100,1.79,CA,1701832126


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
city_data_df["Lat"] = city_data_df["Lat"].astype("float64")
city_data_df["Lng"] = city_data_df["Lng"].astype("float64")

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng",
                                          "Lat",
                                          size = "Humidity",
                                          frame_width = 700,
                                          frame_height = 500,
                                          geo = True,
                                          tiles = "OSM",
                                          color = "City")
# Display the map 
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
fil_df = city_data_df.loc[(city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] <= 5)]


# Drop any rows with null values
fil_df.dropna(inplace = True)

# Display sample data
fil_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,piton saint-leu,-21.2196,55.3151,23.56,73,0,2.57,RE,1701832140
88,88,remire-montjoly,4.9167,-52.2667,25.02,94,0,0.00,GF,1701832189
89,89,port hueneme,34.1478,-119.1951,21.18,57,0,2.06,US,1701832189
105,105,salalah,17.0151,54.0924,22.05,53,0,2.57,OM,1701832200
121,121,timbuktu,16.7735,-3.0074,21.34,12,0,3.72,ML,1701832212
131,131,sao jose da coroa grande,-8.8978,-35.1478,25.23,77,0,4.68,BR,1701832220
294,294,sur,22.5667,59.5289,25.12,41,0,3.83,OM,1701832356
391,391,prado,-17.3411,-39.2208,24.75,88,0,3.68,BR,1701832423
408,408,agbor,6.2518,6.1934,23.38,94,0,1.45,NG,1701832434
429,429,dogondoutchi,13.6393,4.0287,23.48,15,0,4.69,NE,1701832449


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = fil_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,piton saint-leu,RE,-21.2196,55.3151,73,
88,remire-montjoly,GF,4.9167,-52.2667,94,
89,port hueneme,US,34.1478,-119.1951,57,
105,salalah,OM,17.0151,54.0924,53,
121,timbuktu,ML,16.7735,-3.0074,12,
131,sao jose da coroa grande,BR,-8.8978,-35.1478,77,
294,sur,OM,22.5667,59.5289,41,
391,prado,BR,-17.3411,-39.2208,88,
408,agbor,NG,6.2518,6.1934,94,
429,dogondoutchi,NE,13.6393,4.0287,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
base_url = "https://api.geoapify.com/v2/places?"
category = "accommodation.hotel"
params = {
    "categories" : category,
    "limit" : 1,
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        
    except(KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data
hotel_df

Starting hotel search
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
remire-montjoly - nearest hotel: Complexe Belova
port hueneme - nearest hotel: Americas Best Value Inn Oxnard Port Hueneme
salalah - nearest hotel: Muscat International Hotel
timbuktu - nearest hotel: Hotel La Maison
sao jose da coroa grande - nearest hotel: Hotel do Sol
sur - nearest hotel: Sur Hotel
prado - nearest hotel: Pousada Recanto do Prado
agbor - nearest hotel: No hotel found
dogondoutchi - nearest hotel: Maison Alheri
tete - nearest hotel: Zambezi Hotel
chikwawa - nearest hotel: Memories Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
24,piton saint-leu,RE,-21.2196,55.3151,73,Appartement Piton Saint Leu
88,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
89,port hueneme,US,34.1478,-119.1951,57,Americas Best Value Inn Oxnard Port Hueneme
105,salalah,OM,17.0151,54.0924,53,Muscat International Hotel
121,timbuktu,ML,16.7735,-3.0074,12,Hotel La Maison
131,sao jose da coroa grande,BR,-8.8978,-35.1478,77,Hotel do Sol
294,sur,OM,22.5667,59.5289,41,Sur Hotel
391,prado,BR,-17.3411,-39.2208,88,Pousada Recanto do Prado
408,agbor,NG,6.2518,6.1934,94,No hotel found
429,dogondoutchi,NE,13.6393,4.0287,15,Maison Alheri


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]

hotel_df["Lat"] = city_data_df["Lat"].astype("float64")
hotel_df["Lng"] = city_data_df["Lng"].astype("float64")

# Configure the map plot
fig2 = hotel_df.hvplot.points(
                            "Lng",
                            "Lat",
                            geo = True,
                            tiles = "OSM",
                            frame_width = 700,
                            frame_height = 500,
                            color = "City",
                            hover_cols = ["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
    
)

# Display the map
fig2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)